In [2]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa

img_dir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/images')
outdir = Path('/bsuhome/zacharykeskinen/scratch/data/uavsar/coherence')

In [21]:
# for each location directory
for loc_dir in img_dir.glob('*'):

    print(loc_dir)
    # prepare to load cors into this 
    cors = []
    incs = []
    
    # get flight directions
    flight_dirs = np.unique([int(d.stem.split('_')[1][:3]) for d in loc_dir.glob('*_grd')])
    for flight_direction in flight_dirs:
        print(flight_direction)

        flight_dir_inc = None

        # for each image pair directory
        for pair_dir in loc_dir.glob(f'*_{flight_direction}*_grd'):

            # snag incidence angle file
            inc_fp = next(pair_dir.glob('*.inc.tiff'))

            # snag annotation file 
            ann = pd.read_csv(next(pair_dir.glob('*.csv')), index_col = 0)

            for tiff_fp in pair_dir.glob('*cor.grd.tiff'):

                if not flight_dir_inc:
                    # prep and append incidence angle raster
                    inc = rxa.open_rasterio(inc_fp)
                    inc = inc.squeeze('band')
                    inc = inc.coarsen(x = 6, boundary = 'pad').mean().coarsen(y = 6, boundary = 'pad').mean()
                
                polarization = tiff_fp.stem.split('_')[-2][4:]

                if polarization != 'VV':
                    continue

                # open image
                cor = rxa.open_rasterio(tiff_fp)
                
                # remove band as a dimension
                cor = cor.squeeze('band')

                # get metadata frome annotation file or names
                t1 = pd.to_datetime(ann.loc['value', 'start time of acquisition for pass 1'])
                t2 = pd.to_datetime(ann.loc['value', 'start time of acquisition for pass 2'])

                rlooks = int(ann.loc['value', 'number of looks in range'])
                alooks = int(ann.loc['value', 'number of looks in azimuth'])

                # add flight 1,2, direction as dimensions
                cor = cor.expand_dims(time = [t1])
                # cor = cor.assign_coords(direction = ('flight1', [flight_direction]))
                # cor = cor.expand_dims(flight2 = [t2])
                cor = cor.assign_coords(flight2 = ('time', [t2]))
                # cor = cor.expand_dims(polarization = [polarization])

                # coarsen from ~ 5m to ~ 30m
                cor = cor.coarsen(x = 6, boundary = 'pad').mean().coarsen(y = 6, boundary = 'pad').mean()
                rlooks *= 6
                alooks *= 6

                # save appropriate number of looks
                cor.attrs['azimuth_looks'] = alooks
                cor.attrs['range_looks'] = rlooks

                # reproject all to match first image
                if cors:
                    cor = cor.rio.reproject_match(cors[0])

                # append this to our list of coherences
                cors.append(cor)
        
        ds = xr.concat(cors, dim = 'time', combine_attrs = 'identical')
        ds = ds.to_dataset(name = 'vv_coh', promote_attrs = True)
        ds['inc'] = inc.rio.reproject_match(cors[0])

        out_nc = f"{loc_dir.stem.replace(', ','-')}_{flight_direction}_VV.nc"
        ds.to_netcdf(outdir.joinpath(out_nc))

/bsuhome/zacharykeskinen/scratch/data/uavsar/images/Lowman, CO
/bsuhome/zacharykeskinen/scratch/data/uavsar/images/Lowman, CO/lowman_23205_21009-004_21012-000_0007d_s01_L090_01_int_grd/lowman_23205_21009-004_21012-000_0007d_s01_L090VV_01.cor.grd.tiff
/bsuhome/zacharykeskinen/scratch/data/uavsar/images/Lowman, CO/lowman_23205_21015-008_21017-018_0007d_s01_L090_01_int_grd/lowman_23205_21015-008_21017-018_0007d_s01_L090VV_01.cor.grd.tiff
/bsuhome/zacharykeskinen/scratch/data/uavsar/images/Lowman, CO/lowman_23205_21012-000_21015-008_0021d_s01_L090_01_int_grd/lowman_23205_21012-000_21015-008_0021d_s01_L090VV_01.cor.grd.tiff
/bsuhome/zacharykeskinen/scratch/data/uavsar/images/Lowman, CO/lowman_23205_21019-018_21021-006_0006d_s01_L090_01_int_grd/lowman_23205_21019-018_21021-006_0006d_s01_L090VV_01.cor.grd.tiff


KeyboardInterrupt: 